In [1]:
import matplotlib
%matplotlib tk
%autosave 180
%load_ext autoreload
%autoreload 2

import nest_asyncio
%config Completer.use_jedi = False

#
import matplotlib.pyplot as plt
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

# import scipy
import numpy as np
# import pandas as pd
import os

from calcium import calcium
# from wheel import wheel
# from visualize import visualize
# from tqdm import trange

import scipy




Autosaving every 180 seconds


In [54]:
#############################################
######### LOAD SUITE2P AND BINARIZE #########
#############################################

# input directory where Suite2p outputs matlab Fall.mat and all .npy files
data_dir = '/media/cat/4TB/donato/steffen/DON-004366/20210228/'
data_dir = '/media/cat/4TB/donato/steffen/DON-004366/20210301/'
#data_dir = '/media/cat/4TB/donato/DON-003343/DON-003343_20210213/suite2p/plane0/'

c = calcium.Calcium()
c.verbose = True             # outputs additional information during processing
c.recompute = True           # recomputes binarization and other processing steps; False: loads from previous saved locations
c.data_dir = data_dir
c.load_suite2p()

# binarize fluorescence data
c.save_python = True         # save output as .npz file 
c.save_matlab = False         # save output as .mat file

# Tweak these parameters: higher -> less events; lower -> more events
c.min_thresh_std_Fluorescence_onphase = 10
c.min_thresh_std_Fluorescence_upphase = 10

# 
c.binarize_fluorescence()



computing std of global signal (finding max)
Found cells with 0 values (to be removed):  57
  Fluorescence data loading information
         sample rate:  30 hz
         self.F (fluorescence):  (1316, 92900)
         self.Fneu (neuropile):  (1316, 92900)
         self.iscell (Suite2p cell classifier output):  (1962, 2)
         # of good cells:  (1316,)
         self.spks (deconnvoved spikes):  (1316, 92900)
         self.stat (footprints structure):  (1316,)
         mean std over all cells :  20.0

  Binarization parameters: 
        low pass filter low cuttoff:  1 hz
        oasis_thresh_prefilter:  15
        min_thresh_std_oasis:  0.1
        min_thresh_std_Fluorescence_onphase:  10
        min_thresh_std_Fluorescence_upphase:  10
        min_width_event_Fluorescence:  15
        min_width_event_oasis:  2
        min_event_amplitude:  1


low pass filter: 100%|██████████| 1316/1316 [00:01<00:00, 674.13it/s]


computing std of global signal (finding max)
Found cells with 0 values (to be removed):  57
 std global:  6.5


binarizing continuous traces filtered fluorescence onphase: 100%|██████████| 1259/1259 [00:01<00:00, 879.11it/s] 
binarizing continuous traces filtered fluorescence upphase: 100%|██████████| 1259/1259 [00:03<00:00, 383.12it/s] 


   Oasis based binarization skipped by default ... 


In [ ]:
#####################################################
############## LOAD INSCOPIX AND BINARIZE ###########
#####################################################

# input calcium traces .csv file from Inscopix
fname = '/media/cat/4TB/donato/laurenz/DON-005099/20210609/suite2p/plane0/DON-005099_20210609_TRACES_ACQ.csv'

c = calcium.Calcium()
c.verbose = True             # outputs additional information during processing
c.recompute = True           # recomputes binarization and other processing steps; False: loads from previous saved locations
c.data_dir = os.path.split(fname)[0]

# Load inscopix data
c.save_python = True         # save output as .npz file
c.save_matlab = True         # save output as .mat file
c.fname_inscopix = fname
c.load_inscopix()

#
c.binarize_fluorescence()

In [55]:
###############################################################
######### LOAD BINARZIED DATA AND VISUALIZE CELL ACTIVITY #####
###############################################################
#fname = '/media/cat/4TB/donato/DON-003343/DON-003343_20210213/suite2p/plane0/binarized_traces.npz'
#fname = '/media/cat/4TB/donato/steffen/DON-004366/20210228/binarized_traces.npz'
fname = '/media/cat/4TB/donato/steffen/DON-004366/20210301/binarized_traces.npz'

# 
c = calcium.Calcium()
c.fname = fname
c.load_binarization()

# pick a random cell to visuzlie
cell_id = 0

# 
scale = 100

####################################################
fig = plt.figure()
t=np.arange(c.F_filtered.shape[1])/c.sample_rate
plt.plot(t,c.F_filtered[cell_id]/scale, linewidth=3, label='filtered', alpha=.8)
plt.plot(t,c.F_onphase_bin[cell_id]*.9, linewidth=3, label='onphase', alpha=.8)
plt.plot(t,c.F_upphase_bin[cell_id], linewidth=3, label='upphase', alpha=.8)
plt.legend(fontsize=20)
plt.xlabel("Time (sec)")
plt.xlim(t[0], t[-1])
plt.title("Cell: "+str(cell_id))
plt.show()

(1259, 92900)


In [59]:
######################################################
############# SHOW RASTERS ###########################
######################################################
idx = np.where(c.F_onphase_bin==1)

# 
img = np.zeros((c.F_onphase_bin.shape[0], c.F_onphase_bin.shape[1]))
for k in range(idx[0].shape[0]):
    img[idx[0][k],idx[1][k]-5: idx[1][k]+5]=1
    
plt.imshow(img, aspect='auto',
          cmap='Greys')
plt.ylabel("Neuron ID (ordered by SNR by Suite2p)")
plt.xlabel("Time (frames)")
plt.show()
